# UI of indicator lights violation project

### Intro
This is the first php work I've done, and also the first time I got in touch with php language. It is the User Interface of one of the college project I was working on, the "Real-Time Detection of Traffic Violation for Turing without Blinking Indicator Lights" project. It took about a whole week to finish. The final showcase was uploaded on 1st Oct in 2024, in my junior year in CCU. 

### environment
+ XAMPP
    + Apache
    + MySQL

### pre-work before getting started
![mysql](https://raw.githubusercontent.com/HutakiHare/IIS_intern/main/%E8%9E%A2%E5%B9%95%E6%93%B7%E5%8F%96%E7%95%AB%E9%9D%A2%202024-09-29%20135156.png)
+ user_data: where to put info of the user post data
+ user_registration: record info of the registered user
> CREATE TABLE users (  
    id INT AUTO_INCREMENT PRIMARY KEY,  
    username VARCHAR(30) NOT NULL,  
    email VARCHAR(50) NOT NULL,  
    password VARCHAR(255) NOT NULL  
);  

### prerview
![preview](https://raw.githubusercontent.com/HutakiHare/IIS_intern/main/52136d60-5a1e-4b43-8380-6c54aca2bab9.gif)

### register interface
+ register user data
+ send data to save_data.php for processing  

![register](https://raw.githubusercontent.com/HutakiHare/IIS_intern/main/%E8%9E%A2%E5%B9%95%E6%93%B7%E5%8F%96%E7%95%AB%E9%9D%A2%202024-09-28%20155649.png)

```php
<!DOCTYPE html>
<html lang="zh_TW">
<head>
    <meta charset="UTF-8">
    <title>違規偵測系統</title>
    <style>

       /*.. some CSS for better looking ..*/
    
    </style>
</head>
<body>
    <form action="save_data.php" method="POST">
        <h2>註冊帳號</h2>
        <label for="usr_name">使用者名稱:</label>
        <input type="text" id="usr_name" name="usr_name" required><br>
        
        <label for="email_name">電子郵件帳號:</label>
        <input type="email" id="email_name" name="email_name" required><br>
        
        <label for="password">密碼:</label>
        <input type="password" id="password" name="password" required><br>
        
        <label for="double_check_pssword">再次確認密碼:</label>
        <input type="password" id="double_check_psswd" name="double_check_psswd" required><br>
        
        <input type="submit" value="註冊帳號">
        <a href="login.php" class="login-btn">已經有帳號？登入</a>
    </form>
</body>
</html>
```


### save data file
+ check if the user's id, email are used
+ determine if the input data is qualified
+ fill the boxes and add the user to "users" table
+ create a table with the id in "user_data" database and make a dir to collect its videos
+ after registration, turn to login.php  

![process](https://raw.githubusercontent.com/HutakiHare/IIS_intern/main/%E8%9E%A2%E5%B9%95%E6%93%B7%E5%8F%96%E7%95%AB%E9%9D%A2%202024-09-29%20010343.png)

```php
<?php

/*.. connect to sql with $conn ..*/

// 檢查接收到POST請求
if ($_SERVER["REQUEST_METHOD"] == "POST") {
    // 取得表單輸入的值
    $usrname = $_POST['usr_name'];
    $email = $_POST['email_name'];
    $password = $_POST['password'];
    $double_check_psswd = $_POST['double_check_psswd'];

    // 查詢資料庫中的使用者
    $sql_name = "SELECT * FROM users WHERE username='$usrname'";
    $name_result = $conn->query($sql_name);
    $sql_email = "SELECT * FROM users WHERE email='$email'";
    $email_result = $conn->query($sql_email);
    $new_page = "https://localhost/register.php";
    $color = "#FF3333";

    // 資料驗證
    if (empty($usrname) || empty($email) || empty($password) || empty($double_check_psswd)) {
        echo "所有欄位都是必填的！";
    } else if ($password != $double_check_psswd){
        $message = "輸入密碼不同!!\n請重新檢查輸入";
    } else if ($name_result->num_rows > 0){
        $message = '該使用者名稱已經被註冊了';
    } else if ($email_result->num_rows > 0){
        $message = '該電子郵件已經被註冊過了';
    } else {
        // 檢查連接
        $sql = "INSERT INTO users (username, email, password) VALUES ('$usrname', '$email', '$password')";
        if ($conn->query($sql) === TRUE) {
            // 動態切換資料庫
            if ($conn->select_db("user_data")) {
                $createTableQuery = "CREATE TABLE $usrname (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    file_name VARCHAR(255) NOT NULL,
                    file_size INT NOT NULL,
                    upload_date DATETIME DEFAULT NULL,
                    file_type VARCHAR(255) NOT NULL,
                    file_path VARCHAR(255) NOT NULL
                )";
                $directory = 'uploads/' . $usrname; // 這裡輸入要創建的資料夾路徑
                if ($conn->query($createTableQuery) === TRUE && mkdir($directory)) {
                    $message = "新記錄建立成功";
                    $color = "#42ff33";
                    $new_page = "https://localhost/login.php";
                } else {
                    $message = "創建資料庫失敗: " . $conn->error;
                }
            } else {
                die("連接失敗: " . $conn->connect_error);
            }
        } else {
            $message = "錯誤: " . $sql . "<br>" . $conn->error;
        }
        // 關閉連接
        $conn->close();
    }
    // 使用 header() 函數進行重定向
    header("Refresh: 3; URL=$new_page"); // 5秒後自動重定向到登入頁

    ?>
    <!DOCTYPE html>
    <html lang="zh_TW">
    <head>
        <meta charset="UTF-8">
        <style>

          /.. some CSS for better looking ../
        
        </style>
    </head>
    <body>

    <div id="message"><?php echo $message; ?></div> <!-- 顯示成功消息 -->

    <script>

        /*.. some Js for output msg ..*/
    
    </script>

    </body>
    </html>
    <?php

    // 終止腳本的執行
    exit();
}
?>
```

### login interface
+ input id and password to login
+ turn to login_process.php to verify the information  

![login](https://raw.githubusercontent.com/HutakiHare/IIS_intern/main/%E8%9E%A2%E5%B9%95%E6%93%B7%E5%8F%96%E7%95%AB%E9%9D%A2%202024-09-28%20155706.png)

```php
<!DOCTYPE html>
<html lang="zh_TW">
<head>
    <meta charset="UTF-8">
    <title>登入帳號</title>
    <style>

       /*.. some CSS for better looking ..*/
    
    </style>
</head>
<body>
    <form action="login_process.php" method="POST">
    <h2>登入帳號</h2>
        <label for="usr_name">使用者名稱:</label>
        <input type="text" id="usr_name" name="usr_name" required><br>

        <label for="password">密碼:</label>
        <input type="password" id="password" name="password" required><br>

        <input type="submit" value="登入">
    </form>
</body>
</html>
```

### login data interface
+ verify if the id and password are valid
+ if valid, store the username in session  

![login process](https://raw.githubusercontent.com/HutakiHare/IIS_intern/main/%E8%9E%A2%E5%B9%95%E6%93%B7%E5%8F%96%E7%95%AB%E9%9D%A2%202024-09-29%20010400.png)

```php
<?php

/*.. connecct to sql with $conn ..*/

// 開啟 session
session_start();

// 檢查POST資料
if ($_SERVER["REQUEST_METHOD"] == "POST") {
    $usr_name = $_POST['usr_name'];
    $password = $_POST['password'];

    // 查詢資料庫中的使用者
    $sql = "SELECT * FROM users WHERE username='$usr_name'";
    $result = $conn->query($sql);
    $new_page = "https://localhost/login.php";
    $color = "#FF3333";

    if ($result->num_rows > 0) {
        // 取得使用者的資料
        $row = $result->fetch_assoc();

        // 驗證密碼是否正確
        if ($password == $row['password']) {
            // 登入成功，重定向到歡迎頁面
            $message = "登入成功！歡迎, " . $usr_name;
            $new_page = "https://localhost/home.php";
            $color = "#42ff33";
            
            $_SESSION['username'] = $usr_name;
            $_SESSION['selected_file_path'] = NULL;
            $_SESSION['fileSelect'] = NULL;

        } else {
            $message = "密碼錯誤！";
        }
    } else {
        $message = "找不到該使用者！";
    }
    // 使用 header() 函數進行重定向
    header("Refresh: 3; URL=$new_page"); // 3秒後自動重定向到登入頁
    ?>
    <!DOCTYPE html>
    <html lang="zh_TW">
    <head>
        <meta charset="UTF-8">
        <style>

           /.. some CSS for better looking ../
        
        </style>
    </head>
    <body>

    <div id="message"><?php echo $message; ?></div>
    <script>

        /*.. some Js for output msg ..*/
    
    </script>

    </body>
    </html>
    <?php          
    // 終止腳本的執行
    exit();
}

$conn->close();
?>
```

### Home
+ show the instruction of how to use this system
+ with sidebar to go to other pages or logout  

![home](https://raw.githubusercontent.com/HutakiHare/IIS_intern/main/%E8%9E%A2%E5%B9%95%E6%93%B7%E5%8F%96%E7%95%AB%E9%9D%A2%202024-10-01%20120449.png)

```php
<!DOCTYPE html>
<html lang="zh_TW">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>即時車輛偵測違規系統</title>
    <style>
        
        /* 整體佈局 */
       
    </style>
</head>
<body>

<!-- 側邊欄區域 -->
<div class="sidebar">
    <h2>RT-DTV</h2>
    <a href="home.php">Home</a>
    <a href="upload.php">Upload</a>
    <a href=run.php">Run</a>
    <a href="register.php" class = "logout">Logout</a>
</div>

<!-- 內容區域 -->
<div class="content">
    <h1>Real-Time Detection of Traffic Violation</h1>
    <div class="img-container">
        <img src="pictures/system.png" alt="system">
        <img src="pictures/tutorial.png" alt="tutorial">
    </div>
</div>

</body>
</html>
```

### upload
+ choose the mp4 videos to be uploaded (can upload multi files a time)
+ turn to uoload_process.php
+ with sidebar to go to other pages or logout  

![upload](https://raw.githubusercontent.com/HutakiHare/IIS_intern/main/%E8%9E%A2%E5%B9%95%E6%93%B7%E5%8F%96%E7%95%AB%E9%9D%A2%202024-10-01%20120456.png)


```php
<!DOCTYPE html>
<html lang="zh_TW">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>即時車輛偵測違規系統</title>
    <style>
        
        /* 整體佈局 */

    </style>
</head>
<body>

<!-- 側邊欄區域 -->
<div class="sidebar">
    <h2>RT-DTV</h2>
    <a href="home.php">Home</a>
    <a href="upload.php">Upload</a>
    <a href="run.php">Run</a>
    <a href="register.php" class="logout">Logout</a>
</div>

<!-- 內容區域 -->
<div class="content">
    <h1>Real-Time Detection of Traffic Violation</h1>
    <div class = upload-container>
    <form method="post" enctype="multipart/form-data" action="upload_process.php">
        <!-- 限制上傳檔案的最大值 -->
        <!-- <input type="hidden" name="MAX_FILE_SIZE" value="2097152"> -->
        <input type="file" name="my_file[]" accept = "video/mp4" multiple>
        <br><br>
        <input type="submit" value="Upload">
        </form>
    </div>

</div>

</body>
</html>
```

### upload process
+ update the table of the user
+ store the path to the uploaded videos
+ keep the video in a place (in this case, it's uploads/$username/)
+ need a place to temporary keep the videos (in this case, it's uploads/tmp/)
+ verify if the video name already exist, if yes ask user to replace or not
+ replace: turn to replace.php file  

![file store](https://raw.githubusercontent.com/HutakiHare/IIS_intern/main/%E8%9E%A2%E5%B9%95%E6%93%B7%E5%8F%96%E7%95%AB%E9%9D%A2%202024-09-29%20150336.png)

```php
<?php

 /*.. connect to sql with $conn ..*/

// 開啟 session
session_start();
// 設定時區，以獲取正確的當地時間
date_default_timezone_set('Asia/Taipei');

// 檢查接收到POST請求
if ($_SERVER["REQUEST_METHOD"] == "POST") {
    // 取得表單輸入的值
    $files = $_FILES['my_file'];
    $uploadedFiles = [];
    $usrname = $_SESSION['username'];

    foreach ($files['name'] as $index => $fileName) {
        $fileTmpName = $files['tmp_name'][$index];
        $fileSize = $files['size'][$index];
        $upload_date = date('Y-m-d H:i:s');
        $fileType = strtolower($files['type'][$index]);
        
        // 確認檔案是否已存在於資料庫
        $query = "SELECT * FROM $usrname WHERE file_name = ?";
        $stmt = $conn->prepare($query);
        $stmt->bind_param("s", $fileName);
        $stmt->execute();
        $result = $stmt->get_result();
        if ($result->num_rows > 0) {
            // 如果檔案重複，提示用戶選擇是否替換
            $self_tmp_store = 'uploads/tmp/' . basename($fileName);
            move_uploaded_file($fileTmpName, $self_tmp_store);
?>
            <!DOCTYPE html>
            <html lang="zh_TW">
            <head>
                <meta charset="UTF-8">
            </head>
            <body>
                <form id="replaceForm" method="POST" action="replace.php">
                    <input type="hidden" name="fileName" id="fileNameInput">
                    <input type="hidden" name="tmp_name" id="file_tmp_Input" value=<?php echo $self_tmp_store;?>>
                    <input type="hidden" name="fileSize" id="fileSizeInput" value=<?php echo $fileSize; ?>>
                    <input type="hidden" name="fileType" id="fileTypeInput" value=<?php echo $fileType; ?>>
                </form>
            </body>
            </html>
<?php   
            echo "<script>
                document.getElementById('fileNameInput').value = '" . addslashes($fileName) . "';
                if (confirm('檔案 \"" . addslashes($fileName) . "\" 已存在，是否要取代?')) {
                    document.getElementById('replaceForm').submit();
                }else{
                    alert('檔案未被取代。');
                    // Optionally redirect or handle the case where the user cancels
                    window.location.href = 'upload.php'; // Redirect back to upload page
                }
            </script>";
        } else {
            // 如果檔案不重複，則進行上傳
            $uploadPath = 'uploads/' . $usrname . '/' . basename($fileName);
            if (move_uploaded_file($fileTmpName, $uploadPath)) {
                // 將檔案資訊寫入資料庫
                $insertQuery = "INSERT INTO $usrname (file_name, file_size, upload_date, file_type, file_path) VALUES (?, ?, ?, ?, ?)";
                $insertStmt = $conn->prepare($insertQuery);
                $insertStmt->bind_param("sisss", $fileName, $fileSize, $upload_date, $fileType, $uploadPath);
                $insertStmt->execute();
                
                $message = "上傳完成!";
                $color = "#42ff33";
                $uploadedFiles[] = $fileName;
            }else{
                $message = "上傳失敗\n請重新上傳";
                $color = "#FF3333";
            }
        }
    }
    
    $conn->close();
    
    $new_page = 'https://localhost/upload.php';

    // 使用 header() 函數進行重定向
    header("Refresh: 3; URL=$new_page"); // 5秒後自動重定向到登入頁
    
    ?>
    <!DOCTYPE html>
    <html lang="zh_TW">
    <head>
        <meta charset="UTF-8">
        <style>

             /*.. some CSS for better looking ..*/
        
        </style>
    </head>
    <body>

    <div id="message"><?php echo $message; ?></div> <!-- 顯示成功消息 -->

    <script>
         
        /*.. some Js for output msg ..*/
    
    </script>

    </body>
    </html>
    <?php

    // 終止腳本的執行
    exit();
}
?>
```

### replace
+ update the info of the replacing video to mysql (table) and user database
+ turn to upload.php after finished  

![replace](https://raw.githubusercontent.com/HutakiHare/IIS_intern/main/%E8%9E%A2%E5%B9%95%E6%93%B7%E5%8F%96%E7%95%AB%E9%9D%A2%202024-10-01%20222137.png)

```php
<?php

 /*.. connect to sql with $conn ..*/

// 開啟 session
session_start();
// 設定時區，以獲取正確的當地時間
date_default_timezone_set('Asia/Taipei');

if ($_SERVER["REQUEST_METHOD"] == "POST"){
    $fileName = $_POST['fileName']; // 接收檔案名稱
    $usrname = $_SESSION['username'];
    $uploadPath = 'uploads/' . $usrname . '/' . basename($fileName);
    
    if (file_exists($uploadPath)){
        unlink($uploadPath);
    }
    $fileTmpName = $_POST['tmp_name']; // 檔案臨時路徑
    $fixfileTmpName = addslashes($fileTmpName);
    $fileSize = $_POST['fileSize']; // 檔案大小
    $upload_date = date('Y-m-d H:i:s');
    $fileType = $_POST['fileType']; // 檔案類型
    
    if (!file_exists($fileTmpName)) {
        die("臨時檔案不存在: " . $fileTmpName);
    }
    if (empty($uploadPath)) {
        die("上傳路徑未正確設定.");
    }

    if (rename($fileTmpName, $uploadPath)) {
        // Update the database record
        $updateQuery = "UPDATE $usrname SET file_type = ?, file_size = ?, upload_date = ? , file_path = ? WHERE file_name = ?";
        $updateStmt = $conn->prepare($updateQuery);
        $updateStmt->bind_param("sisss",  $fileType, $fileSize, $upload_date, $uploadPath, $fileName);

        if ($updateStmt->execute()) {
            $message = "檔案更新成功!";
        } else {
            $message = "檔案更新失敗: " . $updateStmt->error;
        }

        $updateStmt->close();
    } else {
        $message = "檔案 " . $fileName . " 更新失敗!";
    }
    // Move the uploaded file and replace the old one

    $conn->close();
    $new_page = 'https://localhost/upload.php';
    header("Refresh: 3; URL=$new_page"); // 2秒後自動重定向到上傳頁面
    ?>
    <!DOCTYPE html>
    <html lang="zh_TW">
    <head>
        <meta charset="UTF-8">
        <style>

             /*.. some CSS for better looking ..*/
        
        </style>
    </head>
    <body>

    <div id="message"><?php echo $message; ?></div> <!-- 顯示成功消息 -->

    <script>

        /*.. some Js for output msg ..*/
    
    </script>

    </body>
    </html>
    <?php

    // 終止腳本的執行
    exit();
}
?>
```

### run
+ let user select the video to process
+ show the video after user click "Choose" (turn to run_process.php without displaying) 
+ with sidebar to go to other pages or logout  

![run](https://raw.githubusercontent.com/HutakiHare/IIS_intern/main/%E8%9E%A2%E5%B9%95%E6%93%B7%E5%8F%96%E7%95%AB%E9%9D%A2%202024-10-01%20120524.png)

```php
<!DOCTYPE html>
<html lang="zh_TW">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>即時車輛偵測違規系統</title>
    <style>

        /* 整體佈局 */

    </style>
</head>
<body>

<!-- 側邊欄區域 -->
<div class="sidebar">
    <h2>RT-DTV</h2>
    <a href="home.php">Home</a>
    <a href="upload.php">Upload</a>
    <a href="run.php">Run</a>
    <a href="register.php" class="logout">Logout</a>
</div>

<!-- 內容區域 -->
<?php $show = False; ?>
<div class="content">
    <h1>Real-Time Detection of Traffic Violation</h1>
    <div class = run-container>
        <form action="run_process.php" method="post">
            <select name="fileSelect" id="fileSelect">
                <?php
                session_start();
                $usrname = $_SESSION['username'];

                /*.. connect to sql with $conn ..*/
                
                $sql_name = "SELECT * FROM $usrname";
                $result = $conn->query($sql_name);

                if ($result->num_rows > 0) {
                    while ($row = $result->fetch_assoc()) {
                        echo "<option value='" . htmlspecialchars($row["file_path"]) . "'>" . htmlspecialchars($row["file_name"]) . "</option>";
                    }
                } else {
                    echo "<option value=''>沒有可用的檔案</option>";
                }
                ?>
            </select>
        <br><br>
        <input type="submit" value="Choose">
        </form>

        <!-- 顯示處理後的檔案 -->
        <?php
       
        if (isset($_SESSION['selected_file_path'])) {
            $show = True;
            $file_path = $_SESSION['selected_file_path'];
            echo "<h4>選擇的檔案：</h4>";
            echo "<video width='600' controls>
                    <source src='" . htmlspecialchars($file_path) . "' type='video/mp4'>
                  您的瀏覽器不支援影片播放。
                  </video>";
        
        ?>
        <br>
        <div class = func_button>
            <form action="run_program.php" method="post">
                <input type="submit" value="Run">
                <input type="submit" value="程式影片生成">
                <input type="submit" value="Result">
                <input type="submit" value="Download">
            </form>
        </div>

    <?php
        }
    ?>

    </div>
</div>

</body>
</html>
```

### run process
+ find the selected video, and return to run.php  

![run process](https://raw.githubusercontent.com/HutakiHare/IIS_intern/main/%E8%9E%A2%E5%B9%95%E6%93%B7%E5%8F%96%E7%95%AB%E9%9D%A2%202024-10-01%20120503.png)


```php
<?php
session_start();

if ($_SERVER["REQUEST_METHOD"] == "POST") {
    if (!empty($_POST["fileSelect"])) {
        // 將選擇的檔案路徑存到 session
        $_SESSION['selected_file_path'] = $_POST["fileSelect"];
    }
}

// 回到 run.php 頁面
header("Location: run.php");
exit();
?>
```

### run program
+ act according to the four buttons below video
+ connect to python codes to process the videos  

<span style="color: skyblue;">**To Be Continue**</span>


:::{topic} info
To see full code, please see [my GitHub](https://github.com/HutakiHare/mybook/tree/main/mybook/htdocs)
:::